# PLATO-republic

PLATO will use multiple cameras to observe the same stars. Hopefully the systematics affecting the different cameras will be different. Here I develop an algorithm for modelling the systematics simultaneously with the intrinsic variability of the stars, and disentagling the two, that makes use of this specific configuration.

## The model

The observed (background-subtracted) flux for star $i$ ($i=1 \rightarrow I$) in camera $j$ ($j=1\rightarrow J$) at exposure $k$ ($k=1\rightarrow K$) is modelled as follows:

$$
F_{ijk} = A_{ik} + B_{ijk} + E_{ijk}
$$

where 
* $A_{ik} $ represents the intrinsic flux of star $i$ in frame $k$, 
* $B_{ijk}$ the systematic noise for star $i$ in frame $k$ on camera $j$, and 
* $E_{ijk}$ the corresponding random noise, which we assume to be white and Gaussian with (known) standard deviation $\sigma_{ijk}$

### Caveats

The above model assumes that:
- the systematics are additive rather than multiplicative;
- the contamination and aperture losses are the same in all cameras.

One could account for multiplicative trends by modelling the logarithm of the flux, but one would then have to treat the white noise as multiplicative also which is unlikely to be strictly correct. In practice one might have to try both and see which works best.

The model can be modified to account for variable contamination and aperture losses by replacing $A_{ik}$ with $\alpha_{ij} A_{ik} + \beta{ij}$ where $0<\alpha_{ij} \le 1$ represents the fraction of the total flux of star $i$ captured in the aperture in camera $j$ and $\beta_{ij}$ the contaminating flux in the aperture.

However to keep things simple these modifications are not discussed further here.

### Linear basis model for the systematics

The systematics are modelled as a linear combination of individual "trends" which are specific to each camera but common to all stars on that camera:

$$
B_{ijk} = \sum_{n=1}^{N_j} W_{ijn} T_{jkn}
$$

where $T_{jkn}$ is the value of the $n^{\rm th}$ systematic trend for camera $j$ in frame $k$, $N_j$ is the number of systematic trends for camera $j$, and $w_{ijn}$ is the coefficient linking that trend to star $i$. 

To keep the notation simple, from now on I will assume $N_j=N ~ \forall j$, but the results can easily be extended to the case where different numbers of trends are used for different cameras. From now on I will also write $\sum_k$ instead of $\sum_{k=1}^K$, and so on.

The above equation can be re-written in matrix form:

$$
\mathbf{b}_{ij} = \mathbf{w}_{ij}^T \mathrm{T}_j
$$

where
* $\mathbf{b}_{ij}$ is a $K$-element vector with elements $\{B_{ijk}\}$,
* $\mathbf{w}_{ij}$ is a $N_j$-element vector with elements $\{W_{ijn}\}$, and 
* $\mathrm{T}_{j}$ is a $K \times N$-element matrix with elements $\{T_{jkn}\}$.

Then the overall model becomes

$$
\mathbf{f}_{ij} = \mathbf{a}_i + \mathbf{b}_{ij} + \mathbf{e}_{ij}
$$

where
* $\mathbf{f}_{ij}$ is a $K$-element vector with elements $\{F_{ijk}\}$,
* $\mathbf{a}_i$ is a $K$-element vector with elements $\{A_{ik}\}$,
* $\mathbf{e}_{ij}$ is a $K$-element vector with elements $\{E_{ijk}\}$,


### Two-step modelling approach

The algorithm has two steps:
1. identifying the systematic trends for each camera, and
2. fitting the trends and instrinsic variability for each star. 

This is very much like the PDC-MAP pipeline used by Kepler, K2 and TESS, and indeed the trend identification scheme is basically the same, so I only describe it briefly. 

On the other hand the availability of multiple time-series per star from different cameras makes it possible to fit the trends for each star in a different manner that explicitly models the stellar variability as well, and should hopefully be more robust.

### Trend identification

Following the PDC-MAP approach (Smith et al. 2012), the trends can be identified as the first few principal components of a matrix constructed from the most mutually correlated light curves on a given camera, appropriately weighted. Alternatively (or in addition) external house-keeping parameters such as satellite pointing, detector temperature and camera focus can be included in the basis.

From now on we assume the trends are known.

### Star-by-star fitting

Once the trends are known, each star can be modelled independently so I will drop the $i$ subscript from now on. 

If we model the $J$ observed time-series for each star simultaneously, we have $D \equiv K \times J$ observables, and $P \equiv K + J \times N$ parameters: the $A_k$'s and $W_{jn}$'s (plus optionally a scaling factor or additive term for the white noise). So long as $D \gg P$, the model should be well constrained. 

If the white noise standard deviation is known (which is a fair assumption for space data), maximising the model likelihood w.r.t. the parameters is equivalent to minimizing the total $\chi^2$. As the model is purely linear this should result in a set of simultaneous equations that can be re-written as one big matrix equation.

## $\chi^2$ minimization

### Writing down the $\chi^2$

The total $\chi^2$ for star $k$ is given by

$$
\begin{aligned}
\chi^2 & = \sum_j \sum_k \sigma_{jk}^{-2} \left[ F_{jk} - A_k - \sum_n W_{jn} T_{jkn} \right]^2 \\
& = \sum_j \sum_k \sigma_{jk}^{-2} \left[ F^2_{jk} + A_k^2 + \sum_n W_{jn} T_{jkn} \sum_{m = 1}^N W_{jm} T_{jkm} - 2 F_{jk} A_k + 2 ( A_k - F_{jk}) \sum_n W_{jn} T_{jkn} \right]
\end{aligned}.
$$

### Differentiating the $\chi^2$ w.r.t. the parameters:

To differentiate w.r.t. $A_k$, first write out just the terms involving A_k:

$$
\chi^2(A_k~{\rm terms~only}) = \sum_j \sigma^{-2}_{jk} \left[ A_k^2 - 2 F_{jk} A_k + 2 A_k \sum_n W_{jn} T_{jkn}\right].
$$

Now differentiate:

$$
\frac{\partial \chi^2}{\partial A_k} = 2 A_k \sum_j \frac{1}{\sigma_{jk}^2} - 2 \sum_j \frac{F_{jk}}{\sigma_{jk}^2} + 2 \sum_j \sum_n \frac{W_{jn} T_{jkn}}{\sigma_{jk}^2}
$$

To differentiate w.r.t. $W_{jn}$, first write down just the terms involving $W_{jn}$:

$$
\begin{aligned}
\chi^2(W_{jn}~{\rm terms~only}) & = \sum_k \sigma^{-2}_{jk} \left\{ W_{jn} T_{jkn} \left[ \sum_{m=1}^N  W_{jm} T_{jkm} + 2 (A_k - F_{jk}) \right] \right\} \\
& = \sum_k \sigma^{-2}_{jk} \left[ W^2_{jn} T^2_{jkn} + W_{jn} T_{jkn}\sum_{m \ne n} W_{jm} T_{jkm} + 2 W_{jn} T_{jkn} (A_k - F_{jk}) \right] \\
\end{aligned}
$$

Now differentiate:

$$
\frac{\partial \chi^2}{\partial W_{jn}} = 2 W_{jn} \sum_k \frac{ T_{jkn}^2}{\sigma_{jk}^2} + 2 \sum_{m \ne n} W_{jm}
\sum_k \frac{T_{jkn} T_{jkm}}{\sigma_{jk}^2} + 2 \sum_k \frac{A_k T_{jkn}}{\sigma_{jk}^2} - 2 \sum_k \frac{F_{jk} T_{jkn}}{\sigma_{jk}^2}
$$

### Setting the derivatives to zero

The $\chi^2$ is minimized if:

$$
\begin{aligned}
A_k \sum_j \frac{1}{\sigma_{jk}^2} + \sum_j \sum_n \frac{W_{jn} T_{jkn}}{\sigma_{jk}^2} - \sum_j \frac{F_{jk}}{\sigma_{jk}^2} = 0 & ~ \forall \, k, ~~~ {\rm and}\\
\sum_k \frac{A_k T_{jkn}}{\sigma_{jk}^2} + W_{jn} \sum_k \frac{T_{jkn}^2}{\sigma_{jk}^2} + \sum_{m \ne n} W_{jm} \sum_k \frac{T_{jkn} T_{jkm}}{\sigma_{jk}^2} - \sum_k \frac{F_{jk} T_{jkn}}{\sigma_{jk}^2} = 0 & ~ \forall \, j,n.
\end{aligned}
$$

### Matrix form

The set of equations minimising the $\chi^2$ can be rewritten as a (big) matrix equation $\mathrm{X} \mathbf{p} = \mathbf{y}$, where $\mathbf{p}$ is a $P$-element vector containing all the parameters, while the matrix $\mathrm{X}$ and the vector $\mathbf{y}$ depend only on the $F_{jk}$'s, the $\sigma_{jk}$'s and the $B_{jkn}$'s. 

Once we have defined $\mathrm{X}$ and $\mathbf{y}$, the best-fit parameters $\mathbf{p}$ can be found by numerically solving the matrix equation.

Let us expand $\mathrm{X}$, $\mathbf{p}$ and $\mathbf{y}$. 

$$
\begin{bmatrix}
\mathrm{C} & \mathrm{D}_1 & \mathrm{D}_2 & \dots & \mathrm{D}_J \\
\mathrm{D}_1^T & \mathrm{E}_1 & 0 & \dots & 0 \\
\mathrm{D}_2^T & 0 & \mathrm{E}_{2} & \dots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
\mathrm{D}_J^T & 0 & 0 & \dots & \mathrm{E}_J
\end{bmatrix}
\begin{bmatrix}
\mathbf{a} \\
\mathbf{w}_1 \\
\mathbf{w}_2 \\
\vdots \\
\mathbf{w}_J
\end{bmatrix} 
=
\begin{bmatrix}
\mathbf{q} \\
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_J
\end{bmatrix},
$$

where the sub-matrices on the LHS are given by:

$$
\mathrm{C} = \begin{bmatrix}
\sum_j \sigma_{j1}^{-2} & 0 & \dots & 0 \\
0 & \sum_j \sigma_{j2}^{-2} & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & \sum_j \sigma_{jK}^{-2}
\end{bmatrix},~~~~~
\mathrm{D}_j = \begin{bmatrix}
\frac{T_{j11}}{\sigma_{j1}^2} & \frac{T_{j12}}{\sigma_{j1}^2} & \dots & \frac{T_{j1N}}{\sigma_{j1}^2} \\
\frac{T_{j21}}{\sigma_{j2}^2} & \frac{T_{j22}}{\sigma_{j2}^2} & \dots & \frac{T_{j2N}}{\sigma_{j2}^2} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{T_{jK1}}{\sigma_{jK}^2} & \frac{T_{jK2}}{\sigma_{jK}^2} & \dots & \frac{T_{jKN}}{\sigma_{jK}^2} \\
\end{bmatrix}~~~~~{\rm and}~~~~~
\mathrm{E}_j = \begin{bmatrix}
\sum_k \frac{T_{jk1}^2}{\sigma_{jk}^2} & \sum_k \frac{T_{jk1} T_{jk2}}{\sigma_{jk}^2} & \dots & \sum_k \frac{T_{jk1} T_{jkN}}{\sigma_{jk}^2} \\
\sum_k \frac{T_{jk1} T_{jk2}}{\sigma_{jk}^2} & \sum_k \frac{T_{jk2}^2}{\sigma_{jk}^2} & \dots & 
\sum_k \frac{T_{jk2} T_{jkN}}{\sigma_{jk}^2} \\
\vdots & \vdots & \ddots & \vdots \\
\sum_k \frac{T_{jk1} T_{jkN}}{\sigma_{jk}^2} & \sum_k \frac{T_{jk2} T_{jkN}}{\sigma_{jk}^2} & \dots & 
\sum_k \frac{T_{jkN}^2}{\sigma_{jk}^2}.
\end{bmatrix}
$$

while the sub-vectors on the RHS are given by:

$$
\mathbf{q} = \begin{bmatrix}
\sum_j \frac{F_{j1}}{\sigma_{j1}^2} \\
\sum_j \frac{F_{j2}}{\sigma_{j2}^2} \\
\vdots \\
\sum_j \frac{F_{jK}}{\sigma_{jK}^2} \\
\end{bmatrix}~~~~~{\rm and}~~~~~
\mathbf{r}_j = \begin{bmatrix}
\sum_k \frac{F_{jk} T_{jk1}}{\sigma_{jk}^2} \\
\sum_k \frac{F_{jk} T_{jk2}}{\sigma_{jk}^2} \\
\vdots \\
\sum_k \frac{F_{jk} T_{jkN}}{\sigma_{jk}^2} \\
\end{bmatrix}.
$$

In shorter notation:
* $\mathrm{C} = {\rm diag} \left\{ \sum_j \sigma_{jk}^{-2},~k=1 \rightarrow K\right\}$, 
* the ${kn}^{\rm th}$ element of $\mathrm{D}_j$ is equal to $\tilde{T}_{jkn} \equiv \frac{T_{jkn}}{\sigma_{jk}^2}$,
* the ${nm}^{\rm th}$ element of $\mathrm{E}_j$ is the dot product of the $n^{\rm th}$ and $m^{\rm th}$ rows of $\tilde{\mathrm{T}}_j$, where $\tilde{\mathrm{T}}_j = \mathrm{T}_j / \{ \sigma_{jk} \}$.
* $q_k = \sum_j \frac{F_{jk}}{\sigma_{jk}^2}$ is the variance-weighted average of the observed flux in frame $k$ across all cameras
* the ${n}^{\rm th}$ element of $\mathrm{E}_j$ is the dot product of $\tilde{F}_{jk}$ and the $n^{\rm th}$ row of $\tilde{\mathrm{T}}_j$, where $\tilde{\mathrm{F}}_{jk} = F_{jk}/\sigma_{jk}$.

### Inverting X

As we have broken $\mathbf{X}$ into blocks:

$$
\mathbf{X} =\begin{bmatrix} \mathbf{C}  & \mathbf{D} \\ \mathbf{D}^T & \mathbf{E}\end{bmatrix}
$$

we can write its inverse

$$
\mathbf{X}^{-1} =
\begin{bmatrix} 
\mathbf{C}^{-1} + \mathbf{C}^{-1} \mathbf{D} (\mathbf{E} - \mathbf{D}^T \mathbf{C}^{-1} \mathbf{D})^{-1} \mathbf{D}^T \mathbf{C}^{-1} & 
- \mathbf{C}^{-1}\mathbf{D} (\mathbf{E} - \mathbf{D}^T \mathbf{C}^{-1} \mathbf{D})^{-1} \\ 
- (\mathbf{E}-\mathbf{D}^T \mathbf{C}^{-1} \mathbf{D})^{-1} \mathbf{D}^T \mathbf{C}^{-1} & 
(\mathbf{E} -\mathbf{D}^T \mathbf{C}^{-1} \mathbf{D})^{-1}
\end{bmatrix}
$$

As $\mathbf{C}$ is diagonal, 

$$
\mathbf{C}^{-1}_{k} = 1/\mathbf{C}_{k}=(\sum_j \sigma_{jk}^2)^{-1}.
$$ 